<a href="https://colab.research.google.com/github/CrrazyPeach/GitColab/blob/main/ML/feature_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
data = pd.read_csv(r"/content/drive/MyDrive/Report/anonymized_full_release_competition_dataset.csv",low_memory=False) 
#auto-generate index
#warning: Ln and Ln-1 have mixed dtype('0')
data.head()

,studentId,MiddleSchoolId,InferredGender,SY ASSISTments Usage,AveKnow,AveCarelessness,AveCorrect,NumActions,AveResBored,AveResEngcon,...,RES_CONFUSED,RES_FRUSTRATED,RES_OFFTASK,RES_GAMING,Ln-1,Ln,MCAS,Enrolled,Selective,isSTEM
0,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.000000,0.0,0.785585,0.000264,0.13,0.061190409,45,0,0,NaN
1,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.887452,0.0,0.468252,0.001483,0.061190409,0.213509945,45,0,0,NaN
2,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.887452,0.0,0.468252,0.001483,0.116,0.033305768,45,0,0,NaN
3,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.000000,0.0,0.108417,0.010665,0.116,0.033305768,45,0,0,NaN
4,8,2,Male,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,...,0.000000,0.0,0.108417,0.010665,0.033305768,0.118385889,45,0,0,NaN


In [3]:
data.info()
# features have empty values are:
# 2 InferredGender  769160 non-null (delete empty records)
# 58 sumTime3SDWhen3RowRight  942731 non-null (delete empty records)
# 81 isSTEM  316974 non-null (delete this unclearly defined feature 
#    having many empty values as well as not important - 0.002 mutual information in test)
data=data.drop('isSTEM',axis=1) #delete feature isSTEM

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942816 entries, 0 to 942815
Data columns (total 82 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   studentId                               942816 non-null  int64  
 1   MiddleSchoolId                          942816 non-null  int64  
 2   InferredGender                          769160 non-null  object 
 3   SY ASSISTments Usage                    942816 non-null  object 
 4   AveKnow                                 942816 non-null  float64
 5   AveCarelessness                         942816 non-null  float64
 6   AveCorrect                              942816 non-null  float64
 7   NumActions                              942816 non-null  int64  
 8   AveResBored                             942816 non-null  float64
 9   AveResEngcon                            942816 non-null  float64
 10  AveResConf                              9428

In [ ]:
######Step 1: Transform Ln and Ln-1 from mixed values to floats

In [4]:
import numpy as np

#show invalid records when performing str to float
i=0
arr = []
while i<data.shape[0]:
  try:
    float(data['Ln'][i])
    float(data['Ln-1'][i])
  except:
    arr.append(i)
    print(i,'False',data['Ln'][i])
  i=i+1
 

245078 False .
259371 False .
259372 False .
307392 False .
386368 False .
386369 False .
386370 False .
386371 False .
508746 False .


In [ ]:
#assign value '0' to replace '.'
for number in arr:
  data['Ln'][number] = '0'
  data['Ln-1'][number] = '0' 

#transform to floats
data['Ln'] = data['Ln'].astype(float)
data['Ln-1'] = data['Ln'].astype(float)
data.info()

In [6]:
#Step 2: Tackle records with empty values

In [7]:
#delete all rows have empty values
data.dropna(axis=0,inplace=True)

In [9]:
data.info()
data=data.reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 769102 entries, 0 to 942212
Data columns (total 81 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   studentId                               769102 non-null  int64  
 1   MiddleSchoolId                          769102 non-null  int64  
 2   InferredGender                          769102 non-null  object 
 3   SY ASSISTments Usage                    769102 non-null  object 
 4   AveKnow                                 769102 non-null  float64
 5   AveCarelessness                         769102 non-null  float64
 6   AveCorrect                              769102 non-null  float64
 7   NumActions                              769102 non-null  int64  
 8   AveResBored                             769102 non-null  float64
 9   AveResEngcon                            769102 non-null  float64
 10  AveResConf                              7691

In [ ]:
#Step 3: One-hot decoder shall be introduced to Nominal Variables

In [ ]:
#Nominal Variables are:
#2  InferredGender
#3  SY ASSISTments Usage4
#15  skill
#17  problemType

In [14]:
#extract features to be processed
from sklearn.preprocessing import OneHotEncoder
GenderData = data.iloc[:,2]
SYData = data.iloc[:,3]
skillData = data.iloc[:,15]
problemTypeData = data.iloc[:,17]
noOfencFeature=4

#make sure the proportion of bivalent variable is reasonable after record deletion
(GenderData == 'Male').sum() #407637
(GenderData == 'Female').sum() #361523

#join the features
Tobeenc = pd.concat([GenderData,pd.DataFrame(SYData)],axis=1)
Tobeenc = pd.concat([Tobeenc,pd.DataFrame(skillData)],axis=1)
Tobeenc = pd.concat([Tobeenc,pd.DataFrame(problemTypeData)],axis=1)
Tobeenc
Tobeenc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769102 entries, 0 to 769101
Data columns (total 4 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   InferredGender        769102 non-null  object
 1   SY ASSISTments Usage  769102 non-null  object
 2   skill                 769102 non-null  object
 3   problemType           769102 non-null  object
dtypes: object(4)
memory usage: 23.5+ MB


In [15]:
#fit and transform data using one-hot encoder 
enc = OneHotEncoder(categories='auto').fit(Tobeenc) #fit\
enc.transform(Tobeenc)
result = enc.transform(Tobeenc).toarray() #toarray

#get onehot-feature names
featureNames = enc.get_feature_names_out()
featureNames

#join the result at the end of the original data
data = pd.concat([data,pd.DataFrame(result)],axis=1)

#drop former nominal features
data.drop(["InferredGender","SY ASSISTments Usage","skill","problemType"],axis=1,inplace=True)

data.head()

,studentId,MiddleSchoolId,AveKnow,AveCarelessness,AveCorrect,NumActions,AveResBored,AveResEngcon,AveResConf,AveResFrust,...,111,112,113,114,115,116,117,118,119,120
0,8,2,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,8,2,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,8,2,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,8,2,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,8,2,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [16]:
# featureNames.shape[0] #121 0-120
# data.columns[77] #0

#rename the one-hot encoded features
i=0
while i < featureNames.shape[0]:
  data.rename(columns={i:featureNames[i]},inplace=True)
  i+=1

data.head()


,studentId,MiddleSchoolId,AveKnow,AveCarelessness,AveCorrect,NumActions,AveResBored,AveResEngcon,AveResConf,AveResFrust,...,problemType_interfacepopupmenuquestion1,problemType_interfaceradioquestion,problemType_interfaceradioquestion1,problemType_interfacetextfieldquestion,problemType_interfacetextfieldquestion1,problemType_noprobtype,problemType_other,problemType_popupmenuquestion,problemType_radioquestion,problemType_textfieldquestion
0,8,2,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,8,2,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,8,2,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,8,2,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,8,2,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
#Step 4: generate X,y and analyse with Mutual Information

In [17]:
#randomly take 300000 records
data_=data.sample(n=300000,random_state=2)
X=data_.drop(["correct"],axis=1,inplace=False)
y=data_.iloc[:,19]

#mutual information 
from sklearn.feature_selection import mutual_info_classif as MIC
result = MIC(X,y)
result

array([2.48328110e-02, 1.67322253e-02, 2.35109296e-02, 2.26519042e-02,
       2.38688544e-02, 1.73343723e-02, 2.33067684e-02, 2.46331692e-02,
       2.31723039e-02, 2.47175215e-02, 2.40032602e-02, 2.37224809e-02,
       5.56009923e-02, 5.29430411e-02, 1.16440444e-02, 4.92692633e-02,
       1.94374599e-02, 2.90893895e-02, 3.66072253e-02, 5.84336227e-03,
       2.10051151e-01, 6.58072675e-02, 9.80929527e-02, 6.49791981e-02,
       2.99631753e-02, 1.88080587e-02, 2.58033413e-03, 4.90582479e-03,
       2.57320554e-03, 1.16313955e-03, 3.29158729e-03, 2.06258414e-03,
       2.50108778e-03, 7.84035611e-03, 6.92399788e-03, 1.31114557e-03,
       2.96792414e-03, 3.64513942e-02, 6.90134328e-04, 2.25377377e-03,
       1.86712890e-04, 0.00000000e+00, 1.72181990e-02, 6.32465922e-04,
       7.31241681e-03, 4.84285657e-03, 3.04549727e-03, 1.93357620e-02,
       8.11315272e-04, 3.24371200e-03, 1.76153192e-02, 2.59906460e-01,
       3.78401250e-02, 1.00978509e-01, 1.10037910e-03, 1.39691797e-01,
      

In [18]:
#see nos of irrelevant features
(result == 0).sum()

43

In [19]:
FeatureIndex = pd.DataFrame(X.columns)
weightTable = pd.concat([FeatureIndex,pd.DataFrame(result)],axis=1)
weightTable.columns =["Feature","MutualInformation"]
weightTable_ = weightTable.sort_values(by="MutualInformation")

from google.colab import data_table
data_table.enable_dataframe_formatter()
weightTable_

,Feature,MutualInformation
148,skill_point-plotting,0.000000
105,skill_equation-concept,0.000000
159,skill_reduce-fraction,0.000000
99,skill_comparing-fractions,0.000000
97,skill_circumference,0.000000
...,...,...
51,timeGreater10SecAndNextActionRight,0.259906
68,RES_FRUSTRATED,0.290680
62,confidence(FRUSTRATED),0.291697
60,confidence(CONCENTRATING),0.384669
